In [1]:
from imutils import paths
import numpy as np
import imutils
import cv2
import cv2 as cv
import os
import glob
import pyzbar
from __future__ import print_function
import pyzbar.pyzbar as pyzbar

In [2]:
# Watch Out! square_size is tunable and depend on the chessboard image, I am not exactly sure about this particular value, 
# but all that it changes is scale, so if things look correct, but the dimensions seem too low/big, then this is a good suspect
square_size = 0.027 


# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# Horizontal and vertical are flipped, but lazy. The numbers must be exact, and they are the corners inside, not outside
horizontal = 7 
vertical = 4
objp = np.zeros((horizontal*vertical,3), np.float32)
objp[:,:2] = np.mgrid[0:horizontal,0:vertical].T.reshape(-1,2) * square_size

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('Calibration_Images/*.png')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (horizontal,vertical), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (horizontal,vertical), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(200)
cv.destroyAllWindows()
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
mapx, mapy = cv2.initUndistortRectifyMap(mtx, dist, None, mtx, (1280,720), cv2.CV_32FC1)

In [6]:
square_size = 0.054

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# Horizontal and vertical are flipped, but lazy. The numbers must be exact, and they are the corners inside, not outside
# Watch Out! The fact that these are flipped may have something to do with the switching column business
horizontal = 4 
vertical = 7
objp = np.zeros((horizontal*vertical,3), np.float32)
objp[:,:2] = np.mgrid[0:horizontal,0:vertical].T.reshape(-1,2) * square_size

img = cv.imread('Anchor_1_checkerboard_image.png')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
# Find the chess board corners
ret, corners = cv.findChessboardCorners(gray, (horizontal,vertical), None)
# If found, add object points, image points (after refining them)
if ret == True:
    ret, rvecs, tvecs = cv.solvePnP(objp, corners, mtx, dist)
    correcting_rotation_matrix, _ = cv2.Rodrigues(rvecs)
    correcting_translation_vector = tvecs.T
    print('Extracted correcting features for the anchor')

Extracted correcting features for the anchor


In [4]:
def change_coordinate_system(x, y, z):
    x -= correcting_translation_vector[0, 0]
    y -= correcting_translation_vector[0, 1]
    z -= correcting_translation_vector[0, 2]
    row1 = x * correcting_rotation_matrix[0, 0] + y * correcting_rotation_matrix[1, 0] + z * correcting_rotation_matrix[2, 0]
    row2 = x * correcting_rotation_matrix[0, 1] + y * correcting_rotation_matrix[1, 1] + z * correcting_rotation_matrix[2, 1]
    row3 = x * correcting_rotation_matrix[0, 2] + y * correcting_rotation_matrix[1, 2] + z * correcting_rotation_matrix[2, 2]
    shiftingX = correcting_translation_vector[0, 0] * correcting_rotation_matrix[0, 0] + correcting_translation_vector[0, 1] * correcting_rotation_matrix[0, 1] + correcting_translation_vector[0, 2] * correcting_rotation_matrix[0, 2]
    shiftingY = correcting_translation_vector[0, 0] * correcting_rotation_matrix[1, 0] + correcting_translation_vector[0, 1] * correcting_rotation_matrix[1, 1] + correcting_translation_vector[0, 2] * correcting_rotation_matrix[1, 2]
    shiftingZ = correcting_translation_vector[0, 0] * correcting_rotation_matrix[2, 0] + correcting_translation_vector[0, 1] * correcting_rotation_matrix[2, 1] + correcting_translation_vector[0, 2] * correcting_rotation_matrix[2, 2]
    x = row1 #+ shiftingX
    y = row2 #+ shiftingY
    z = row3
    return np.array([x, y, z]).T

In [7]:
qrDecoder = cv2.QRCodeDetector()
cam = cv2.VideoCapture('Anchor_1_video.mp4') 
cv2.namedWindow("output", cv2.WINDOW_NORMAL)

objp = np.zeros((2*2,3), np.float32)   # The order is top left, top right, bottom left, bottom right. img points must match
objp[:,:2] = np.mgrid[0:2,0:2].T.reshape(-1,2) * 0.4   # Tune this value by checking with IRL measurements, was 0.245 for A4, 0.4 for our QR code

while True:
    ret, frame = cam.read()
    if not ret:
        print("End of Video/Coudln't grab frames")
        break
    frame = cv2.remap(frame, mapx, mapy, cv2.INTER_LINEAR, cv2.BORDER_CONSTANT)
 
    decodedObjects = pyzbar.decode(frame)
    for idx, decodedObject in enumerate(decodedObjects):
        points = decodedObject.polygon
        for j in range(4):
            cv2.line(frame, points[j], points[((j+1)%4)], (255,0,0), 3)
        points = np.array(decodedObject.polygon, dtype=int)
#         temp = [[[points[0, 0], points[0, 1]]], [[points[3, 0], points[3, 1]]], [[points[1, 0], points[1, 1]]], [[points[2, 0], points[2, 1]]]]
#         temp = np.array(temp, dtype=np.float32)
#         ret,rvecs, tvecs = cv.solvePnP(objp, temp, mtx, dist)
#         # Watch Out! While testing on my own, I realized that there was something weird, where moving back and forth changed 
#         # the x and y measurement, even though you would expect it to not when only the distance is changing, so I divided it 
#         # by the z value and that kept it constant. However, this also doesn't seem right, and perhaps it is just supposed to do
#         # that due to the way egocentric coordinates work.
#         x = tvecs[0] #/ tvecs[2]   
#         y = tvecs[1] #/ tvecs[2]
#         z = tvecs[2]
        x = 0
        y = 0
        z = 0
        for i in range(4):        
            temp = [[[points[(0 + i)%4, 0], points[(0 + i)%4, 1]]], [[points[(3 + i)%4, 0], points[(3 + i)%4, 1]]], [[points[(1 + i)%4, 0], points[(1 + i)%4, 1]]], [[points[(2 + i)%4, 0], points[(2 + i)%4, 1]]]]
            temp = np.array(temp, dtype=np.float32)
            ret,rvecs, tvecs = cv.solvePnP(objp, temp, mtx, dist)
            x += tvecs[0]/4
            y += tvecs[1]/4
            z += tvecs[2]/4
        # If you run this line instead of the bootom one, it will display the egocentric coordinates on the video feed
#         cv2.putText(frame, str(tvecs) + ' Data: ' + (decodedObject.data).decode('utf-8'), (20, (i + 1) * 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
        # Watch Out! This is the function that you need to implement to align the data
        global_coordinates = change_coordinate_system(x, y, z)
        cv2.putText(frame, str(global_coordinates) + ' Data: ' + (decodedObject.data).decode('utf-8'), (20, (idx + 1) * 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
    
    
    frame = cv2.resize(frame, (1280, 720))
    cv2.imshow("output", frame)   
    k = cv2.waitKey(1)
    if k%256 == 27:   # Stops when ESC is entered
        break
    elif k%256 == 32: # Spacebar to skip a few frames
        for i in range(200):
            cam.read()
cam.release()
cv2.destroyAllWindows()